In [1]:
import os
import sys
import polars as pl
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
import numpy as np
import scipy as sp
import sklearn
import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("display.float_format", '{:.3f}'.format)
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import polars.selectors as cs
import plotly.express as px
from sklearn.decomposition import PCA    
%load_ext autoreload
%autoreload 2
import statsmodels.api as sm

from IPython.display import display
from IPython.display import Javascript, Markdown

In [2]:
try:
    from tdt.data.obb_data import get_treasury_yields, add_bond_metrics_to_obb_data
    from tdt.data.charting import plot_sensitivities, format_weights,format_returns,plot_heat_map, plot_returns
    from tdt.app.widgets import get_layout
except:
    # In case you forgot to pip install
    import os
    import sys
    current_file = os.path.abspath(os.getcwd())

    # Go up 1 or 2 parent levels if needed
    parent1 = os.path.dirname(current_file)
    parent2 = os.path.dirname(parent1)
    # Add all subdirectories of both parents
    for parent in [parent1, parent2]:
        for root, dirs, files in os.walk(parent):
            if root not in sys.path:
                sys.path.append(root)
    
    
    from tdt.data.obb_data import get_treasury_yields, add_bond_metrics_to_obb_data
    from tdt.data.charting import plot_sensitivities, format_weights,format_returns,plot_heat_map, plot_returns
    from tdt.app.widgets import get_layout
    


In [3]:
# Reasonable defaults
schemes=['MV',"DV01","PCA","Pure_LongPC3","Regression"]
valid_legs=['2','5','7','10','20','30']
scheme= schemes[0]

In [4]:
legs=['2','5','10']
long_belly= 1
start_date='2019-01-01'
end_date='2025-01-01'
date_changed=False

In [5]:
start,end,drop,sel,conf,go,layout = get_layout(valid_legs,schemes)
def start_change(*args):
    global start_date
    global date_changed
    start_date = f"{pd.to_datetime(args[0]['new']):%Y-%m-%d}"
    date_changed=True
    
def end_change(*args):
    global end_date
    global date_changed
    end_date = f"{pd.to_datetime(args[0]['new']):%Y-%m-%d}"  
    #print(end_date)
    date_changed=True
    
def wt_change(*args):
    global legs
    l =args[0]['new']
    if len(l)==3:
        legs= args[0]['new']
        #print(legs)

def scheme_change(*args):
    global scheme
    scheme = args[0]['new']
    

def ls_change(*args):
    global long_belly
    long_belly = 1 if args[0]['new'] else -1
    #print(long_belly)
def run_all(*args):
    print('Running')
    display(Javascript('IPython.notebook.execute_cells_below()'))

    
start.observe(start_change,names='value')    
end.observe(end_change,names='value')    
drop.observe(scheme_change,names='value')    
sel.observe(wt_change,'value')
conf.observe(ls_change,'value')
go.on_click(run_all)

In [28]:
layout

<IPython.core.display.Javascript object>

In [29]:
data = None
if date_changed or data is None:
    # do not get data again if date has not changed
    data = add_bond_metrics_to_obb_data(get_treasury_yields('2019-01-01','2025-01-01'))
    date_changed=False

In [30]:
rates=data.with_columns(pl.col('par_yield').mul(100).alias('rate'))\
.pivot(on='term',index='dt',values='rate').to_pandas().set_index('dt').resample('W').last()

In [31]:
diff_rates = rates.diff().dropna()
diff_rates_std = diff_rates-diff_rates.mean()

In [32]:
#To dynamically allocate flies
durations=data.pivot(on='term',index='dt',values='dv01').to_pandas().set_index('dt').iloc[0,:]
indices={v:k for k,v in enumerate(diff_rates)}

In [33]:
display(Markdown(f"## Running {'-'.join(legs)} fly from {start_date} to {end_date} with {scheme} weights"))

## Running 2-5-10 fly from 2019-01-01 to 2025-01-01 with Regression weights

In [34]:
#Run PCA
pca = PCA(n_components=3).fit(diff_rates_std)
pcs= pd.DataFrame(pca.transform(diff_rates_std),columns=['level','slope','curve'],index = diff_rates_std.index)
pc_exposures=pd.DataFrame(pca.components_,columns=diff_rates.columns,index=['level','slope','curve'])

In [48]:
#Extract Raw yield, curve & butterfly to measure sensitivity to changes for regression
endog=(rates[legs[1]]- (0.5*rates[legs[0]] +0.5*rates[legs[2]])).rename("-".join(legs)+' fly')


exog= pcs.iloc[:,:2] #Note no need to diff this as this is already from diff frame
fit=sm.OLS(endog.diff().dropna(),exog,missing='drop').fit()

In [49]:
#Init weights
mvs= np.zeros(len(diff_rates.columns))
dv01s= np.zeros(len(diff_rates.columns))
pca_wts= np.zeros(len(diff_rates.columns))
regression_wts= np.zeros(len(diff_rates.columns))

valid_indices= [indices[leg] for leg in legs]
valid_durations = [durations[leg] for leg in legs]
#static weights
mvs[valid_indices] = np.array([-0.5,1,-0.5])*long_belly
dv01s[valid_indices] = np.array([-0.5*valid_durations[1]/valid_durations[0],1,-0.5*valid_durations[1]/valid_durations[2]])*long_belly

#regression weights
a,b=fit.params
regression_wts[valid_indices]= np.array([-(0.5+b)/(1-a),1,-(0.5-b)/(1-a)])*long_belly

#Pure pca weights
w=(pc_exposures.loc['curve',:]/diff_rates_std.std())
wts =w/np.linalg.norm(w,1)

In [50]:
#PCA weights with 3 assets
opt=pc_exposures.loc[['level','slope'],legs].T
b_=opt.loc[legs[1],:]
A= opt.loc[[l for l in legs if l!=legs[1]],:]
a,b=np.linalg.solve(A,b_)
pca_wts[valid_indices] = np.array([-a,1,-b])*long_belly

In [51]:
if scheme=='Regression':
    display(Markdown('### Regression Weights'))

    display(fit.summary())

### Regression Weights

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             2-5-10 fly   R-squared (uncentered):                   0.202
Model:                            OLS   Adj. R-squared (uncentered):              0.197
Method:                 Least Squares   F-statistic:                              39.32
Date:                Wed, 28 May 2025   Prob (F-statistic):                    5.99e-16
Time:                        22:26:44   Log-Likelihood:                          731.06
No. Observations:                 313   AIC:                                     -1458.
Df Residuals:                     311   BIC:                                     -1451.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
level          0.0386      0.004      8.622      0.000       0.030       0.047
slope          0.0284      0.014      2.073      0.039       0.001       0.055
==============================================================================
Omnibus:                       31.757   Durbin-Watson:                   2.162
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              145.811
Skew:                           0.185   Prob(JB):                     2.18e-32
Kurtosis:                       6.323   Cond. No.                         3.06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
#Changes to diff rates
mv_fly_changes=np.dot(diff_rates_std,mvs)
dv01_fly_changes=np.dot(diff_rates_std,dv01s)
pca_fly_changes=np.dot(diff_rates_std,pca_wts)
pure_pca_changes=np.dot(diff_rates_std,wts)
regression_changes=np.dot(diff_rates_std,regression_wts)
changes=pd.DataFrame([mv_fly_changes,dv01_fly_changes,pca_fly_changes,pure_pca_changes,regression_changes],index=schemes).T

In [40]:
frame=pd.DataFrame([mvs,dv01s,pca_wts,wts,regression_wts],index=schemes,columns=diff_rates.columns)
frame=frame.assign(dv01= np.dot(frame,durations.T),
                   net_notional_mio = frame.iloc[:,:-1].sum('columns')*100
                  )
display(Markdown('### All Weights'))
format_weights(frame)

### All Weights

,2,5,7,10,20,30,dv01,net_notional_mio
MV,-50%,100%,0%,-50%,0%,0%,"-5,600.0",0.0
DV01,-117%,100%,0%,-27%,0%,0%,0.0,-44.2
PCA,-76%,100%,0%,-23%,0%,0%,"11,938.6",1.3
Pure_LongPC3,26%,-17%,-19%,-9%,11%,18%,"24,897.2",-7.8
Regression,-55%,100%,0%,-49%,0%,0%,"-5,785.5",-4.0


In [41]:
fig=plot_sensitivities(changes[scheme].values,pcs,scheme)


In [42]:
derived_rates=pd.DataFrame([np.dot(rates[frame.columns[:-2]],frame.iloc[i,:-2].T) for i in range(len(frame))],columns=rates.index,index= frame.index).T
derived_rates.loc[:,'-'.join(legs)]=(endog*long_belly)

In [43]:
fig=px.line(derived_rates,title=f"Implied {'-'.join(legs)} fly rates by weighting")
fig.update_layout(
        height=400, width=1000,
        template='plotly_white',
        showlegend=True
    )

In [44]:
corr_matrix= derived_rates.diff().corr()
plot_heat_map(corr_matrix,'Corr in daily changes')

In [45]:
#diff then mul by -100 to get bps change &
#get rally/sell off sign right, multiply by dv01 to get 1bp PnL, divide by belly notional
rets=pd.concat([rates.diff().mul(-100).dropna().mul(durations.T).dot(frame.iloc[w,:-2]).div(1e8) for w in range(len(frame))],axis='columns')\
    .rename(columns={k:v for k,v in enumerate(frame.index)})


In [46]:
display(Markdown('### In sample returns'))
format_returns(rets)

### In sample returns

,Mean,Std Dev,Skew,Kurtosis,Annual Return,Annual Volatility,Ann IR,MDD
MV,0.00%,0.16%,0.21,1.59,0.17%,1.14%,0.15,-2.56%
DV01,-0.00%,0.12%,0.10,2.26,-0.05%,0.86%,-0.05,-1.69%
PCA,-0.01%,0.22%,0.16,0.90,-0.40%,1.59%,-0.25,-5.56%
Pure_LongPC3,-0.01%,0.31%,-0.15,0.77,-0.75%,2.26%,-0.33,-9.11%
Regression,0.00%,0.15%,0.18,1.74,0.17%,1.11%,0.15,-2.39%


In [47]:
plot_returns(rets,'Cumulative returns (% of belly notional)')